In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas
import os

In [2]:
path = r'/Users/polinarozhkova/Desktop/GitHub/cr_eda_chicago/'

In [3]:
district_shp = os.path.join(path, 'PoliceDistrict', 'PoliceDistrict.shp')
df_district = geopandas.read_file(district_shp)
df_district['DIST_NUM'] = df_district['DIST_NUM'].astype(float)

In [4]:
beat_shp = os.path.join(path, 'Boundaries - Police Beats (current)',
                        'geo_export_7f053540-59bf-4d25-b453-b6f10b77c115.shp')
beat_df = geopandas.read_file(beat_shp)
beat_df['beat_num'] = beat_df['beat_num'].astype(float)

In [5]:
# Additional data: Homicides, Clearance Rates, Racial Demographics
final_merge_df = pd.read_csv(os.path.join(path, 'clean_data/merge_all.csv'))
cr_reports = pd.read_excel(os.path.join(path, 'inputs/CR_from_CPD_Annual_Reports_copy.xlsx'))
beats_race = pd.read_csv(os.path.join(path, 'inputs/beat_estimates.csv'))

In [6]:
# Racial Demographics by Beat
race_2021 = beats_race[beats_race['year'] == 2021].drop(
    columns='Unnamed: 0').reset_index(drop=True)
race_2021['perc_white'] = round((race_2021.white_nothisp/race_2021.tot_pop)*100, 2)
race_2021['perc_black'] = round((race_2021.black_nothisp/race_2021.tot_pop)*100, 2)
race_2021['perc_hisp'] = round((race_2021.tot_hisp/race_2021.tot_pop)*100, 2)

In [7]:
final_merge_df

,case_number,unique_id,date_x,block_x,primary_type,iucr,age,sex,race,month,...,year,id,date,injury_type,cleared,date_clear,beat,district,time_to_clear,year_cleared
0,G011117,HOM-G011117-#1,2001-01-06 03:30:00,8700 S BURLEY AVE,HOMICIDE,110,20-29,M,BLK,1,...,2001,644.0,2001-01-06 00:00:00,SHOT,Y,2001-02-27,424.0,4.0,52.0,2001.0
1,G011288,HOM-G011288-#1,2001-01-06 08:54:00,1700 S ASHLAND AVE,HOMICIDE,110,20-29,M,BLK,1,...,2001,639.0,2001-01-06 00:00:00,SHOT,N,NaN,1222.0,12.0,NaN,NaN
2,G032407,HOM-G032407-#1,2001-01-16 01:17:00,600 N CICERO AVE,HOMICIDE,110,20-29,M,BLK,1,...,2001,659.0,2001-01-16 00:00:00,SHOT,Y,2001-05-28,1111.0,11.0,132.0,2001.0
3,G040822,HOM-G040822-#1,2001-01-20 01:09:00,1000 N CICERO AVE,HOMICIDE,110,0-19,M,BLK,1,...,2001,663.0,2001-01-20 00:00:00,SHOT,Y,2001-01-24,1111.0,11.0,4.0,2001.0
4,G042611,HOM-G042611-#1,2001-01-20 23:26:00,700 N DRAKE AVE,HOMICIDE,110,30-39,M,WWH,1,...,2001,664.0,2001-01-20 00:00:00,SHOT,Y,2001-07-05,1121.0,11.0,166.0,2001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,JE258289,HOM-JE258289-#1,2021-06-08 12:58:00,3800 S WELLS ST,HOMICIDE,110,20-29,M,BLK,6,...,2021,25982.0,2021-06-08 12:58:00,SHOT,N,NaN,915.0,9.0,NaN,NaN
11495,JD443510,HOM-JD443510-#1,2020-11-27 20:23:00,3900 W JACKSON BLVD,HOMICIDE,110,20-29,M,BLK,11,...,2020,25639.0,2020-11-27 20:23:00,SHOT,N,NaN,1132.0,11.0,NaN,NaN
11496,HK505802,HOM-HK505802-#1,2004-06-19 12:01:00,6900 S CAMPBELL AVE,HOMICIDE,110,20-29,M,BLK,6,...,2004,2806.0,2004-06-19 00:00:00,SHOT,N,NaN,832.0,8.0,NaN,NaN
11497,JC378474,HOM-JC378474-#2,2019-08-04 17:46:00,5400 S WINCHESTER AVE,HOMICIDE,110,50-59,M,BLK,8,...,2019,24678.0,2019-08-04 17:46:00,SHOT,N,NaN,932.0,9.0,NaN,NaN


In [8]:
def clean(df):
    df = df.rename(columns={'ward_x': 'ward', 'beat': 'beat_num','district': 'DIST_NUM'})
    df['cleared'] = df['cleared'].map({'Y': 1, 'N': 0})
    df['gunshot_injury'] = final_merge_df['gunshot_injury_i'].map({'YES': True, 'NO': False})
    return df

In [9]:
final_merge_df = clean(final_merge_df)

In [10]:
def district_grouped(df, df_cleared, df_district):
    hom_district = df.groupby(['DIST_NUM'])['case_number'].count().reset_index()
    hom_district = hom_district.rename(columns={'case_number': 'homicide_count'})
    clear_district = df_cleared.groupby(['DIST_NUM'])['cleared'].sum().reset_index()
    hom_district = hom_district.merge(clear_district['cleared'], on=['DIST_NUM'])
    df_district = df_district.merge(hom_district, on=['DIST_NUM'])
    df_district = df_district.to_crs('EPSG:3435')
    return df_district

In [11]:
def beat_grouped(df, df_cleared, beat_df):
    hom_beat = df.groupby(['beat_num'])['case_number'].count().reset_index()
    hom_beat = hom_beat.rename(columns={'case_number': 'homicide_count'})
    clear_beat = df_cleared.groupby(['beat_num'])['cleared'].sum().reset_index()
    hom_beat = hom_beat.merge(clear_beat['cleared'], on=['beat_num'])
    hom_race_merge = pd.merge(hom_beat, race_2021, how="inner", on=['beat_num'])
    beat_df = beat_df.merge(hom_race_merge, on=['beat_num'])
    beat_df = beat_df.to_crs('EPSG:3435')
    return beat_df

In [12]:
hom_2019 = final_merge_df[final_merge_df['year'] == 2019]
hom_2020 = final_merge_df[final_merge_df['year'] == 2020]
hom_2021 = final_merge_df[final_merge_df['year'] == 2021]

In [13]:
clear_2019 = final_merge_df[final_merge_df['year_cleared'] == 2019]
clear_2020 = final_merge_df[final_merge_df['year_cleared'] == 2020]
clear_2021 = final_merge_df[final_merge_df['year_cleared'] == 2021]

,case_number,unique_id,date_x,block_x,primary_type,iucr,age,sex,race,month,...,id,date,injury_type,cleared,date_clear,beat_num,DIST_NUM,time_to_clear,year_cleared,gunshot_injury
1917,JC366242,HOM-JC366242-#2,2019-07-26 23:35:00,700 N WALLER AVE,HOMICIDE,110,20-29,M,BLK,7,...,24666.0,2019-07-26 23:35:00,SHOT,0,NaN,1511.0,15.0,NaN,NaN,True
5196,JC225720,HOM-JC225720-#3,2019-04-15 17:15:00,0 W 114TH ST,HOMICIDE,110,20-29,M,BLK,4,...,24488.0,2019-04-15 17:15:00,SHOT,0,NaN,522.0,5.0,NaN,NaN,True
7246,JC107385,HOM-JC107385-#1,2019-01-06 22:22:00,6000 W 63RD ST,HOMICIDE,110,20-29,M,WWH,1,...,24372.0,2019-01-06 22:22:00,SHOT,1,2019-05-27,812.0,8.0,140.0,2019.0,True
7265,JC103046,HOM-JC103046-#1,2019-01-03 12:54:00,5000 S INDIANA AVE,HOMICIDE,110,20-29,M,BLK,1,...,24369.0,2019-01-03 12:54:00,SHOT,0,NaN,224.0,2.0,NaN,NaN,True
7266,JC103046,HOM-JC103046-#2,2019-01-03 12:54:00,5000 S INDIANA AVE,HOMICIDE,110,20-29,M,BLK,1,...,24369.0,2019-01-03 12:54:00,SHOT,0,NaN,224.0,2.0,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11392,JC327983,HOM-JC327983-#1,2019-06-29 23:13:00,6000 S KOMENSKY AVE,HOMICIDE,110,0-19,M,WWH,6,...,24625.0,2019-06-29 23:13:00,SHOT,0,NaN,813.0,8.0,NaN,NaN,True
11442,JC415562,HOM-JC415562-#2,2019-09-01 02:47:00,6100 S ST LAWRENCE AVE,HOMICIDE,110,20-29,M,BLK,9,...,24724.0,2019-09-01 02:47:00,SHOT,0,NaN,313.0,3.0,NaN,NaN,True
11458,JC360346,HOM-JC360346-#2,2019-07-22 21:02:00,11400 S LOWE AVE,HOMICIDE,110,0-19,M,BLK,7,...,24659.0,2019-07-22 21:02:00,SHOT,0,NaN,2233.0,22.0,NaN,NaN,True
11462,JC198417,HOM-JC198417-#3,2019-03-25 00:40:00,11700 S LAFLIN ST,HOMICIDE,110,30-39,F,BLK,3,...,24447.0,2019-03-25 00:40:00,SHOT,0,NaN,524.0,5.0,NaN,NaN,True


In [14]:
hom_2019.groupby(['beat_num'])['cleared'].sum().reset_index()

,beat_num,cleared
0,111.0,1
1,114.0,0
2,132.0,1
3,211.0,0
4,212.0,0
...,...,...
172,2531.0,3
173,2532.0,1
174,2533.0,0
175,2534.0,0


In [15]:
hom_2019.groupby(['beat_num'])['case_number'].count().reset_index()

,beat_num,case_number
0,111.0,1
1,114.0,1
2,132.0,1
3,211.0,1
4,212.0,1
...,...,...
172,2531.0,3
173,2532.0,3
174,2533.0,1
175,2534.0,2


In [ ]:
hom_dist_2019 = district_grouped(hom_2019, clear_2019, df_district)
hom_dist_2020 = district_grouped(hom_2020, clear_2020, df_district)
hom_dist_2021 = district_grouped(hom_2021, clear_2021, df_district)

In [ ]:
hom_beat_2019 = beat_grouped(hom_2019, clear_2019, beat_df)
hom_beat_2020 = beat_grouped(hom_2020, clear_2020, beat_df)
hom_beat_2021 = beat_grouped(hom_2021, clear_2021, beat_df)

In [ ]:
hom_dist_2021